In [1]:
import time
import re
import random
random.seed(2016)

import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import pipeline, grid_search
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer

In [2]:
dtrain = xgb.DMatrix("train.buffer")
dtest = xgb.DMatrix("test.buffer")
evallist  = [(dtrain,'train')]

In [3]:
loc = '%s'
#loc = '/home/ec2-user/data/hd/features/%s'
a_o = np.load(loc % 'train_data.npy')
b_o = np.load(loc % 'test_data.npy')
a_brand = np.load(loc % 'features_brand_01_train.npy')
b_brand = np.load(loc % 'features_brand_01_test.npy')
a_other = np.load(loc % 'FEATURES_1d_TRAIN.npy')
b_other = np.load(loc % 'FEATURES_1d_TEST.npy')
#a_word_feat = np.load(loc % 'SPECIAL_WORDS_FEAT_TRAIN.npy')
#b_word_feat = np.load(loc % 'SPECIAL_WORDS_FEAT_TEST.npy')

a_w2vdot = pd.read_pickle(loc % 'W2V_dots_train.df').drop('relevance', axis=1).values
b_w2vdot = pd.read_pickle(loc % 'W2V_dots_test.df').drop('relevance', axis=1).values

#a_w2vdist = np.load(loc % 'W2V_dists_train.npz')['arr_0']
#b_w2vdist = np.load(loc % 'W2V_dists_test.npz')['arr_0']

#a_w2v_el = np.load(loc % 'W2V_vecs_train.npz')['arr_0']
#b_w2v_el = np.load(loc % 'W2V_vecs_test.npz')['arr_0']

In [4]:
a_o.shape, a_brand.shape, a_other.shape, a_word_feat.shape, aa.shape, a_w2vdot.shape, a_w2vdist.shape, a_w2v_el.shape  # 73 #5  #111 #79 #6 #2 #2 #900

NameError: name 'a_word_feat' is not defined

In [17]:
idx_train = pd.read_pickle(loc % 'LABELS_TRAIN.df')
idx_test = pd.read_pickle(loc % 'LABELS_TEST.df')

validation_idx = pd.read_csv(loc % 'valid_set.csv.gz', index_col= 'id').index
train_idx = idx_train.index.difference(validation_idx)

df_a = pd.DataFrame(a, index=idx_train.index)

train_idx.shape, validation_idx.shape

((53907,), (20160,))

In [20]:
a.shape, idx_train.shape

((74067, 191), (74067, 1))

In [32]:
df_training_alex = df_a.loc[train_idx]
df_validating_alex = df_a.loc[validation_idx]

df_training_alex_label = idx_train.loc[train_idx]
df_validating_alex_label = idx_train.loc[validation_idx]

In [34]:
#df_training_alex.shape, df_validating_alex.shape
#idx_train.loc[train_idx]
#df_training_alex_label
df_validating_alex_label

,relevance
id,
2,3.00
16,2.33
18,3.00
35,3.00
69,1.00
88,1.33
113,2.00
117,2.67
123,3.00


In [28]:
df_validating_alex.head()
#df_training_alex.head()

,0,1,2,3,4,5,6,7,8,9,...,181,182,183,184,185,186,187,188,189,190
id,,,,,,,,,,,,,,,,,,,,,
2,100001.0,15.0,2.0,6.0,114.0,3.0,13.0,33.0,735.0,18.0,...,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.191196,0.102050
16,100005.0,25.0,3.0,13.0,81.0,1.0,16.0,73.0,556.0,5.0,...,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.113482,0.081475
18,100006.0,48.0,2.0,12.0,373.0,1.0,14.0,77.0,2538.0,9.0,...,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.293396,0.120656
35,100011.0,42.0,4.0,17.0,196.0,1.0,26.0,107.0,1309.0,4.0,...,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.188877,0.114863
69,100017.0,25.0,2.0,12.0,70.0,3.0,11.0,73.0,450.0,20.0,...,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.047049,0.079902


In [ ]:
73 - basic features, counts, ratios
5 - brand-related
111 - variations of above, with completely different typo cleaning
79 - special words in query, title or description that seemed to correlate with difference in relevance
6 - (dangerous - best removed to avoid leak) depends on split, tries to extract information from 
similar words - counts word co-occurences between query and title/description (a bit like simplified ALS)
2 - dot product between word2vec of query and title/descr
2 - Euclidean distance between word2vec of query and title/descr
900 - full averaged word2vec vectors for query, title, desc

In [ ]:
# LOCAL CV
aaa = pd.read_pickle(loc % 'WOQTAL_TRAIN_147')
bbb = pd.read_pickle(loc % 'WOQTAL_TEST_147')
#aa = aaa.drop('relevance', axis=1).values
#bb = bbb.drop('relevance', axis=1).values
aa = aaa.drop('relevance', axis=1).values[:,[0,3]]
bb = bbb.drop('relevance', axis=1).values[:,[0,3]]

#a = np.hstack((a_o, a_brand, a_other, a_word_feat, a_w2vdot, a_w2vdist, a_w2v_el[:,:700]))
#b = np.hstack((b_o, b_brand, b_other, b_word_feat, b_w2vdot, b_w2vdist, b_w2v_el[:,:700]))

In [ ]:
#a = np.hstack((a_o, a_brand, a_other, a_word_feat, aa, a_w2vdot, a_w2vdist, a_w2v_el))
#b = np.hstack((b_o, b_brand, b_other, b_word_feat, bb, b_w2vdot, b_w2vdist, b_w2v_el))
a = np.hstack((a_o, a_brand, a_other, a_word_feat, aa, a_w2vdot, a_w2vdist, a_w2v_el[:,:700]))
b = np.hstack((b_o, b_brand, b_other, b_word_feat, bb, b_w2vdot, b_w2vdist, b_w2v_el[:,:700]))

In [ ]:
a = np.hstack((a_o, a_brand, a_other, a_w2vdot, a_w2vdist))
b = np.hstack((b_o, b_brand, b_other, b_w2vdot, b_w2vdist))

In [5]:
a = np.hstack((a_o, a_brand, a_other, a_w2vdot))
b = np.hstack((b_o, b_brand, b_other, b_w2vdot))

In [ ]:
locd = '/Users/rbekbolatov/data/kaggle/homedepot/%s'

In [ ]:
a_w2v_el = np.load(loc % 'W2V_vecs_train.npz')['arr_0']

In [ ]:
wmd_train = pd.read_csv(locd % 'wmd_gn_train.csv.gz').values[:,1]
wmd_test = pd.read_csv(locd % 'wmd_gn_test.csv.gz').values[:,1]

In [ ]:
wmd_train[np.where(np.isinf(wmd_train))[0]] = 99

In [ ]:
wmd_train[np.where(np.isinf(wmd_train))[0]]

In [ ]:
#a = np.vstack((a_o.T, a_brand.T, a_other.T, a_word_feat.T, aa.T, a_w2vdot.T, a_w2vdist.T)).T
#a = np.vstack((a_o.T, a_brand.T, a_other.T, a_word_feat.T, aa.T, a_w2vdot.T, a_w2vdist.T, wmd_train, a_w2v_el[:,:700].T)).T
#a = np.vstack((a_o.T, a_brand.T, a_other.T, a_word_feat.T, aa.T, a_w2vdot.T, a_w2vdist.T, a_w2v_el[:,:700].T)).T

#X_train, X_test, y_train, y_test = train_test_split(np.hstack([pawel_train.values,a]), dtrain.get_label(), test_size=0.20, random_state=147) # 0.20, 147
X_train, X_test, y_train, y_test = train_test_split(df_training_alex.values(), dtrain.get_label(), test_size=0.20, random_state=147) # 0.20, 147
gX_train = xgb.DMatrix(data=X_train, label=y_train)
#gX_test = xgb.DMatrix(data=X_test[0:10814,:], label=y_test[0:10814])
gX_test = xgb.DMatrix(data=X_test[10814:,:], label=y_test[10814:])
evallist  = [(gX_train,'train'),(gX_test,'test')]

In [39]:
X_train = df_training_alex.values
y_train = df_training_alex_label.values

X_test = df_validating_alex.values
y_test = df_validating_alex_label.values

X_train.shape, y_train.shape,  X_test.shape, y_test.shape

((53907, 191), (53907, 1), (20160, 191), (20160, 1))

In [40]:
clf = linear_model.Ridge (alpha = 0.6)
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
y_hat = np.minimum(np.maximum(y_hat, 1.0), 3.0)
fmean_squared_error(y_hat, y_test)

#0.47733228483742429

0.47733228483742429

In [41]:
a.shape, X_train.shape, y_train.shape, y_test.shape

((74067, 191), (53907, 191), (53907, 1), (20160, 1))

In [ ]:
X_test[0:10814,:].shape

In [ ]:
wmd_train.sum()

In [ ]:
wmd_train.shape

In [ ]:
a.shape

In [ ]:
a = np.vstack((a_o.T, a_brand.T, a_other.T, a_word_feat.T, a_w2vdot.T, a_w2vdist.T, wmd_train, wmd_train)).T

In [ ]:
a = np.vstack((a_o.T, a_brand.T, a_other.T, a_w2vdot.T, a_w2vdist.T, wmd_train, wmd_train)).T

In [ ]:
#FINAL GENERATE
aaa = pd.read_pickle(loc % 'WOQTAL_TRAIN_ALL')
bbb = pd.read_pickle(loc % 'WOQTAL_TEST_ALL')
aa = aaa.drop('relevance', axis=1).values
bb = bbb.drop('relevance', axis=1).values

a_full = np.hstack((a_o, a_brand, a_other, a_word_feat, aa, a_w2vdot, a_w2vdist, a_w2v_el[:,:700]))
b_full = np.hstack((b_o, b_brand, b_other, b_word_feat, bb, b_w2vdot, b_w2vdist, b_w2v_el[:,:700]))

### DATA SETS READY

In [ ]:
a_full = np.hstack((a_o, a_brand, a_other, a_word_feat, aa, a_w2vdot, a_w2vdist))  #, a_w2v_el))
b_full = np.hstack((b_o, b_brand, b_other, b_word_feat, bb, b_w2vdot, b_w2vdist))  #, b_w2v_el))


a_f = pd.DataFrame(a_full)
b_f = pd.DataFrame(b_full)
a_f[0] = a_f[0].astype(int)
b_f[0] = b_f[0].astype(int)
a_f.to_csv('ALL_SET_TRAIN.csv', index=False)
b_f.to_csv('ALL_SET_TEST.csv', index=False)

In [ ]:
a_w2v = pd.DataFrame(a_w2v_el)
b_w2v = pd.DataFrame(b_w2v_el)
a_w2v.to_csv('W2V_TRAIN.csv', index=False)
b_w2v.to_csv('W2V_TEST.csv', index=False)

In [ ]:
a_f.head()

In [ ]:
queries

In [ ]:
pd.DataFrame(b_full).to_csv('ALL_SET_TEST.csv')

In [ ]:
a.shape

In [ ]:
aaaa = pd.read_csv('Pawel_train_after_processing_withfeatures.csv', index_col='id').sort_index()
bbbb = pd.read_csv('Pawel_test_after_processing_withfeatures.csv', index_col='id').sort_index()

In [ ]:
aaaa.to_pickle('Pawel_train.df')
bbbb.to_pickle('Pawel_test.df')

In [ ]:
pawel_train = aaaa.drop(["product_title", "search_term", "product_description",
                                               "atr_text", 'brand_text', 'color_text', 'material_text',
                                               'bullet01_text', 'bullet02_text', 'bullet03_text', 'bullet04_text', 'bullet05_text', 'bullet06_text',
                                               'bullet07_text', 'bullet08_text', 'bullet09_text', 'bullet10_text', 'bullet11_text', 'bullet12_text', 
           'relevance', 
                         #'product_uid'
                        ], axis=1)
pawel_test = bbbb.drop(["product_title", "search_term", "product_description",
                                               "atr_text", 'brand_text', 'color_text', 'material_text',
                                               'bullet01_text', 'bullet02_text', 'bullet03_text', 'bullet04_text', 'bullet05_text', 'bullet06_text',
                                               'bullet07_text', 'bullet08_text', 'bullet09_text', 'bullet10_text', 'bullet11_text', 'bullet12_text', 
           #'product_uid'
                       ], axis=1)

In [ ]:
pawel_train = ptr.drop(["product_title", "search_term", "product_description",
                                               "atr_text", 'brand_text', 'color_text', 'material_text',
                                               'bullet01_text', 'bullet02_text', 'bullet03_text', 'bullet04_text', 'bullet05_text', 'bullet06_text',
                                               'bullet07_text', 'bullet08_text', 'bullet09_text', 'bullet10_text', 'bullet11_text', 'bullet12_text', 
           'relevance', 
                         #'product_uid'
                        ], axis=1)
pawel_test = pte.drop(["product_title", "search_term", "product_description",
                                               "atr_text", 'brand_text', 'color_text', 'material_text',
                                               'bullet01_text', 'bullet02_text', 'bullet03_text', 'bullet04_text', 'bullet05_text', 'bullet06_text',
                                               'bullet07_text', 'bullet08_text', 'bullet09_text', 'bullet10_text', 'bullet11_text', 'bullet12_text', 
           #'product_uid'
                       ], axis=1)

In [ ]:
pawel_train.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.hstack([pawel_train.values,a]), dtrain.get_label(), test_size=0.20, random_state=147) # 0.20, 147
gX_train = xgb.DMatrix(data=X_train, label=y_train)
gX_test = xgb.DMatrix(data=X_test, label=y_test)
evallist  = [(gX_train,'train'),(gX_test,'test')]

In [ ]:
X_train.shape

In [8]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(a, dtrain.get_label(), test_size=0.20, random_state=147) # 0.20, 147
gX_train = xgb.DMatrix(data=X_train, label=y_train)
gX_test = xgb.DMatrix(data=X_test, label=y_test)
evallist  = [(gX_train,'train'),(gX_test,'test')]

In [ ]:
#clf = linear_model.LinearRegression(n_jobs=8)
#clf = linear_model.Lasso(alpha=0.01)
clf = linear_model.Ridge (alpha = 0.6)
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
y_hat = np.minimum(np.maximum(y_hat, 1.0), 3.0)
fmean_squared_error(y_hat, y_test)  #0.45631070577528032
# 0.45290360003958696
# 0.45286703214175722

# 0.45358284891316947
# 0.45355346051674278
#  0.00003


# 0.45290360003958696
# 0.45273208794146252

In [ ]:

idx_train = pd.read_pickle(loc % 'LABELS_TRAIN.df')


In [ ]:
a_o.T

In [ ]:
#a = np.hstack((a_o, a_brand, a_other, a_word_feat, aa, a_w2vdot, a_w2vdist, a_w2v_el[:,:700]))
a = np.vstack((np.array(idx_train.index), a_o.T, a_brand.T, a_other.T, a_word_feat.T, aa.T, a_w2vdot.T, a_w2vdist.T, a_w2v_el[:,:700].T)).T
a.shape # (74067, 978)

In [ ]:
np.array(idx_train.index)

### Checked LR

In [ ]:
def next_num_round():
    return np.random.randint(750, 950)

num_round = 10000

# TRAIN AS SELF
param = {'max_depth':9, 
         'eta':0.1, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.3,
         'subsample':0.9,
         'nthread':8,
         'silent': True
        }
# GOOD <<
param = {'max_depth':7, 
         'eta':0.03, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.7, #7
         'subsample':0.9,  #8
         'min_child_weight': 4.0,
         'nthread':32,
         'silent': True
        }
param = {'max_depth':9, 
     'eta':0.01, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.7, #7
     'subsample':0.9,  #8
     'min_child_weight': 4.0,
     'nthread':32,
     'silent': True
    }
param = {'max_depth':6, 
     'eta':0.01, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.6, #7
     'subsample':0.8,  #8
     'min_child_weight': 5.0,
     'nthread':32,
     'silent': True,
     'seed': 37
    }

param = {'max_depth':7, 
     'eta':0.01, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.6, #7
     'subsample':0.8,  #8
     'min_child_weight': 5.0,
     'nthread':32,
     'silent': True,
     'seed': 39
    }

num_round = 200000
bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], early_stopping_rounds=100, verbose_eval=10)
#bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], verbose_eval=10)

In [ ]:
# 700x trees  -  cv: 0.4377 (seed 147)
param = {'max_depth':9, 
         'eta':0.01, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.3, #0.8 #7  0.4449
         'subsample':0.8, #0.9,  #8
         'min_child_weight': 4.0,
         'nthread':32,
         'silent': True
        }  
num_round = 5000
bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], early_stopping_rounds=30, verbose_eval=10)

In [ ]:
ggX_train = xgb.DMatrix(data=a_full, label=dtrain.get_label())
ggX_test = xgb.DMatrix(data=b_full)

idx_train = pd.read_pickle(loc % 'LABELS_TRAIN.df')
idx_test = pd.read_pickle(loc % 'LABELS_TEST.df')

num_round = 1550
bst = xgb.train( param, ggX_train, num_round, [(gX_test,'test')], verbose_eval=50)

y_pred = bst.predict(ggX_test)
y_pred_bounded = np.minimum(np.maximum(y_pred, 1.0), 3.0)
idx_test['relevance'] = y_pred_bounded
idx_test.to_csv('submission_RenatPawel_combined_features_xgboost_0408_0137.csv')

In [ ]:
!gzip submission_xgboost_words_0405_1646.csv

In [ ]:
# 1500 trees 
param = {'max_depth':9, 
         'eta':0.01, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.8, #0.8 #7  0.4449
         'subsample':0.9, #0.9,  #8
         'min_child_weight': 4.0,
         'nthread':32,
         'silent': True
        }  
num_round = 5000
bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], early_stopping_rounds=15, verbose_eval=10)

with 0.30 and 1479
cv:  0.4493

In [ ]:
loc = '%s'

aaa = pd.read_pickle(loc % 'WOQTAL_TRAIN_ALL')
bbb = pd.read_pickle(loc % 'WOQTAL_TEST_ALL')


In [ ]:
aaa.head()

In [ ]:
aaa[aaa['woqt1'] > 0.127]['relevance'].mean(), aaa[aaa['woqt1'] < 0.127]['relevance'].mean()

In [ ]:
aaa[aaa['woqt2'] > 0.329895]['relevance'].mean(), aaa[aaa['woqt2'] < 0.329895]['relevance'].mean()

In [ ]:
aaa['woqt2'].mean()

In [ ]:
bbb.head()

In [ ]:
aa = aaa.drop('relevance', axis=1)
bb = bbb.drop('relevance', axis=1)

In [ ]:
aa.reset_index().to_csv('word_co_train.csv', header=True, index=False)

In [ ]:
bb.reset_index().to_csv('word_co_test.csv', header=True, index=False)

In [4]:
pd.read_csv('word_co_validate_local.csv.gz')

,id,woqt1,woqt2,woqt3,woqt4,woqt5,woqt6
0,2,0.054131,0.139973,0.017308,0.033680,0.084147,0.010375
1,16,0.101627,0.259178,0.012521,0.081595,0.214880,0.014222
2,18,0.206146,1.042023,0.161459,0.010576,0.050057,0.007427
3,35,0.250737,0.654225,0.064001,0.181117,0.467228,0.040137
4,69,0.020699,0.052253,0.003647,0.023631,0.042638,0.000358
5,88,0.020683,0.054795,0.001254,0.036444,0.118722,0.000749
6,113,0.025676,0.090455,0.001537,0.011935,0.035702,0.000550
7,117,0.111331,0.442529,0.000667,0.010946,0.034239,0.000162
8,123,0.077544,0.163294,0.000314,0.127381,0.194105,0.000086
9,136,0.139412,0.254918,0.022218,0.136884,0.201258,0.031163
